In [15]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

predictors = pd.read_csv("../../data/variables.csv")
predictors.head(5)

,var_code,var_desc,isTabulated,unique_values,missing_pct
0,uniqueid,UNIQUEID,False,27976,0.000000
1,year,YEAR,True,8,0.000000
2,personid,RESPONDENT_ID_NUMBER,False,3497,0.000000
3,x1,CURRENT_ECONOMIC_SITUATION_IN_GERMANY,True,5,0.004187
4,x2,RESP._OWN_CURRENT_FINANCIAL_SITUATION,True,5,0.001369


In [10]:
#plt.hist(predictors['missing_pct'])
sum(predictors['missing_pct'] > 0.4)

1064

# Data Cleanup Steps

1. 91% missing data threshold
2. remove all `YEAR_OF_BIRTH` variables
3. remove all `INTERVIEWER:` variables
4. `WORK CONDITIONS:` -> Assume Missing implies NO
5. Any variables with (40% <= missing <= 91%) get indicator variables
6. Scale all numerical values to be between 0 and 1
7. kNN Imputation for rest of the variables

In [16]:
# step 1
predictors = predictors[predictors["missing_pct"] < 0.8]
# step 2
predictors = predictors[~predictors["var_desc"].str.contains("YEAR_OF_BIRTH")]
# step 3
predictors = predictors[~predictors["var_desc"].str.contains("INTERVIEWER:")]
# step 4
# df_temp = pd.DataFrame({"isWorkCond": predictors["var_desc"].str.contains("WORK_CONDITIONS")})
# predictors = pd.concat([predictors, df_temp], axis = 1)
# step 5
df_temp = pd.DataFrame({"isIndicator": predictors["missing_pct"] > 0.4})
predictors = pd.concat([predictors, df_temp], axis = 1)

predictors = predictors[0:-1]

In [114]:
predictors

,var_code,var_desc,isTabulated,unique_values,missing_pct,isIndicator
0,uniqueid,UNIQUEID,False,27976,0.000000,False
1,year,YEAR,True,8,0.000000,False
2,personid,RESPONDENT_ID_NUMBER,False,3497,0.000000,False
3,x1,CURRENT_ECONOMIC_SITUATION_IN_GERMANY,True,5,0.004187,False
4,x2,RESP._OWN_CURRENT_FINANCIAL_SITUATION,True,5,0.001369,False
...,...,...,...,...,...,...
1204,x1201,REGION_OF_INTERVIEW:_WEST_-_EAST,True,2,0.000000,False
1205,x1202,FEDERAL_STATE_THAT_RESPONDENT_LIVES_IN,False,17,0.000000,False
1206,x1203,POLITICAL_SIZE_OF_MUNICIPALITY,True,7,0.005032,False
1207,x1204,BIK-REGIONS,False,10,0.016023,False


In [11]:
data_train = pd.read_csv("../../data/train.csv")
data_test = pd.read_csv("../../data/test.csv")

data_test['health'] = np.NaN

In [14]:
sum(data_train.any(axis=1))
len(data_train)

17398

In [116]:
def intersection(lst1, lst2):
    lst3 = [value for value in lst1 if value in lst2]
    return lst3

imp = pd.read_csv("../randomforest/importance_w_desc.csv")


indicators = list(predictors.loc[predictors['isIndicator'],'var_code'])
    
# print(len(indicators))
#     #ret_df = ret_df.drop(columns=indicators)
#     #ret_df[indicators] = ret_df[indicators].notnull().astype('int')
#     #ret_df[protected] = ret_df[protected].notnull().astype('int')
# protected = intersection(predictors['var_code'], indicators)
# #ret_df = data_train[['uniqueid'] + protected]
# 'x633' in protected
temp = data_train[intersection(imp['vars'], list(predictors['var_code']))]
temp

,x633,x1144,x1145,x1203,x725,x1152,x728,x1151,x1148,x730,...,x652,x941,x1161,x940,x909,x942,x646,x638,x1162,x643
0,30.0,3,20040301,1.0,300.0,41.0,1300.0,17.56,17.15,500.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0,0.0
1,63.0,3,20040301,3.0,750.0,41.0,1450.0,17.56,17.15,725.0,...,0.0,NaN,0,NaN,0.0,NaN,0.0,1.0,0,0.0
2,74.0,3,20040301,3.0,1100.0,36.0,1100.0,19.10,18.34,1100.0,...,0.0,NaN,0,NaN,NaN,NaN,0.0,1.0,0,0.0
3,70.0,3,20040301,1.0,1300.0,38.0,1300.0,18.01,17.23,1300.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0,0.0
4,78.0,3,20040313,5.0,1400.0,59.0,1400.0,17.53,16.54,1400.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17393,35.0,7,20160711,4.0,1250.0,59.0,2750.0,20.54,19.55,1528.0,...,0.0,NaN,0,NaN,0.0,NaN,0.0,1.0,0,0.0
17394,48.0,5,20160509,6.0,670.0,80.0,2200.0,16.30,15.10,1467.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,2.0,0,0.0
17395,53.0,5,20160531,3.0,1200.0,36.0,1200.0,17.36,17.00,1200.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0,0.0
17396,75.0,8,20160829,3.0,1200.0,55.0,1200.0,20.55,20.00,1200.0,...,0.0,0.0,0,0.0,0.0,0.0,0.0,1.0,0,0.0


In [6]:
from sklearn.impute import KNNImputer
from sklearn.preprocessing import MinMaxScaler

def gnumeric_func (data, columns):
  data[columns] = data[columns].apply(lambda x: pd.factorize(x)[0])
  return data

def preprocessor(predictors: pd.DataFrame, df: pd.DataFrame, y = 'health', n_neighbours = 20) -> pd.DataFrame:
    # of two lists in most simple way
    def intersection(lst1, lst2):
        lst3 = [value for value in lst1 if value in lst2]
        return lst3
 
    ret_df = df.filter(items = predictors['var_code'], axis = 1)

    # replace NAs of work conditions with 3
    workcond_vars = predictors.loc[predictors['var_desc'].str.contains("WORK_CONDITIONS"), 'var_code']
    ret_df[workcond_vars] = ret_df[workcond_vars].fillna(value=3)

    temp_col = ret_df[y]
    ret_df = ret_df.drop(columns=y)

    ret_df = ret_df.drop('personid', axis=1)

    # normalize btwn 0 and 1
    print("NORMALIZING")
    ret_df.iloc[:,2:] = MinMaxScaler().fit_transform(ret_df.iloc[:,2:])

    # set indicators
    indicators = list(predictors.loc[predictors['isIndicator'],'var_code'])
    
    print(len(indicators))
    #ret_df = ret_df.drop(columns=indicators)
    ret_df[indicators] = ret_df[indicators].notnull().astype('int')
    #ret_df[protected] = ret_df[protected].notnull().astype('int')
    # imp = pd.read_csv("../randomforest/importance_w_desc.csv")
    # protected = intersection(imp['vars'], list(predictors['var_code']))
    # ret_df = ret_df[['uniqueid'] + protected]
    
    # kNN imputer
    print("IMPUTING")
    print(ret_df.isna().sum(axis=1).sum())
    ret_df.iloc[:,:] = KNNImputer(n_neighbors=n_neighbours).fit_transform(ret_df.iloc[:,:])

    ret_df.insert(0, y, temp_col)

    return ret_df

temp_df = preprocessor(predictors, data_test)
temp_df.to_csv("preprocesed_variables_test.csv", index = False)
# temp_df = preprocessor(predictors, data_train)
# temp_df.to_csv("preprocesed_variables_train.csv", index = False)

NORMALIZING
293
IMPUTING
376833


E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3444: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  exec(code_obj, self.user_global_ns, self.user_ns)


In [31]:
temp_df
#temp_df['health'] = np.NaN

#predictors['isTabulated'] & ~(predictors['isIndicator'])

,health,uniqueid,year,x1,x2,x3,x4,x5,x6,x7,...,x1196,x1197,x1198,x1199,x1200,x1201,x1202,x1203,x1204,x1205
0,2,20053999,2005,1.00,0.50,1.00,0.75,0.0,0.0,0.444444,...,0.0,0.0,0.0,0.0,0.0,1.0,1.000000,0.000000,0.333333,0.0
1,3,20053997,2005,0.50,0.50,0.50,0.50,0.0,0.0,0.483333,...,0.0,0.0,0.0,0.0,0.0,1.0,0.866667,0.333333,0.444444,0.0
3,3,20053995,2005,0.50,0.25,0.50,0.50,0.0,0.0,0.444444,...,0.0,0.0,0.0,0.0,0.0,1.0,0.933333,0.000000,0.444444,0.0
4,3,20053993,2005,0.75,0.25,0.75,0.75,0.0,0.0,0.111111,...,0.0,0.0,0.0,0.0,0.0,1.0,0.800000,0.666667,0.777778,0.0
5,3,20053992,2005,0.75,0.50,0.75,0.50,0.0,0.0,0.555556,...,0.0,0.0,0.0,0.0,0.0,0.0,0.133333,0.000000,0.666667,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17391,3,2017516,2017,0.50,0.25,0.50,0.50,0.0,0.0,0.222222,...,1.0,1.0,1.0,1.0,1.0,0.0,0.266667,0.833333,0.777778,1.0
17392,2,2017515,2017,0.50,0.25,0.75,0.50,0.0,0.0,0.444444,...,1.0,1.0,1.0,1.0,1.0,0.0,0.533333,0.166667,0.333333,1.0
17393,2,2017514,2017,0.50,0.50,0.50,0.50,0.0,0.0,0.222222,...,1.0,1.0,1.0,1.0,1.0,0.0,0.333333,0.500000,0.888889,1.0
17394,4,2017513,2017,0.50,0.50,0.75,0.75,0.0,0.0,0.433333,...,0.0,1.0,1.0,1.0,1.0,0.0,0.466667,0.833333,0.777778,1.0
